# Practical: Artificial Intelligence in Python

The objective of this tutorial is to familiarize you with Artificial Intelligence in a case of semantic segmentation of point clouds.The point clouds used were acquired with Velodyne64 and are available from SemanticKITTI.

A semantic segmentation problem consists of classifying each basic element that makes up the data. In image, it is pixels, in point clouds, it is points. Sometimes semantic segmentation is also called pixel-by-pixel or point-by-point classification.


# Objectives

In this tutorial we will look at code snippets that explain how to prepare data for an AI algorithm, train it and use it to make predictions. The operations that will be studied next are:

- Reading and writing point clouds as txt data.
- Data preparation for Artificial Intelligence
- Feature extraction
- Algorithm training
- Analysis of the results using metrics: confusion, precision, recall, f1-score and accuracy matrices
- Classification visualization

# Libraries

The algorithms used in artificial intelligence are quite complex. Although we can program an algorithm with simple knowledge of the subject, getting it to reach the hit rates of algorithms already developed by other authors requires a strong background in mathematics and computer science, as well as powerful test servers. Fortunately, most AI algorithms are open source and compiled in libraries.

In this tutorial we will use the *pyntcloud*, *scikit-learn* and *numpy* libraries.

The first task is to install these libraries in our environment. Once the library is installed, we are going to import all the functions we need. If any of them give error, check that the librerias are correctly installed in the environment.


In [ ]:
import numpy as np
from pyntcloud import PyntCloud
import pandas as pd
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Reading and writing of point clouds as txt data

AI-based algorithms need a training phase before they can be used to classify new data. As input data we are going to use the point clouds 000036 (train) and 000079 (test). We are  going to load point clouds as txt, since this type of format, together with csv, is the most common in which we can find other types of data for AI. The txt cloud is structured in 1 point per row and 1 attribute per column, and ' ' is specified as delimiter between columns.

In [ ]:
# Data reading
train_data = np.loadtxt("Nubes/000036.txt", delimiter=' ')

# Visualization
print(train_data)

The organization of data in AI is distributed in matrices where each row corresponds to a sample and each column to an attribute. This is very similar to how point clouds are distributed. In the above data we can see that each point contains 3 coordinates and the fourth column corresponds to the label according to the following code:
- 1: car
- 2: building
- 3: ground
- 4: vegetation

# Data preparation for Artificial Intelligence

But this data cannot be used directly in an AI algorithm, it must be divided into an NxM attribute matrix, where N is the number of samples and M is the number of attributes, and a label matrix (Nx1).

Furthermore, in point clouds, coordinates cannot be used as training attributes for AI, since the position of points in absolute coordinates does not represent or relate to new data. In point clouds, coordinates are used to extract new geometric features that are suitable for the classifier.

Therefore, first of all, let's divide the input matrix into two matrices:
- Coordinate matrix (defined as a column-titled dataframe, necessary for its transformation to Pyntcloud object).
- Label array (defined as numpy array)

In [ ]:
# Extract coordinate matrix
coord = pd.DataFrame(list(zip(train_data[:,0],train_data[:,1],train_data[:,2])))  

# Assign title to columns
coord.columns =['x', 'y', 'z']

# Visualization
print(coord)

In [ ]:
# Extract label matrix
train_labels = train_data[:,3]

# Visualization
print(train_labels)

# Feature extraction

As mentioned above, coordinates are not useful for training an AI-based algorithm, and therefore we have to extract geometric features. Although we can compute them using the matrix directly, it is easier to convert the matrix to a cloud object in the pyntcloud library and use its functions to compute them. 

For a correct conversion, cloud does not support a numpyarray, but a dataframe with the titles "x", "y", and "z", calculated in the previous step, has to be used.

The features used for training will be:
- Normals
- Curvature
- Omnivariate
- Linearity
- Planarity
- Scattering

The use of these characteristics is widely extended in point clouds, their calculation is explained in the following scientific paper:
- Weinmann, M., Jutzi, B., & Mallet, C. (2014). Semantic 3D scene interpretation: A framework combining optimal neighborhood size selection with relevant features. ISPRS Annals of the Photogrammetry, Remote Sensing and Spatial Information Sciences, 2(3), 181.

In [ ]:
# Conversion
cloud = PyntCloud(coord)

# Calculation of 25 neighbors
k_neighbors_25 = cloud.get_neighbors(k=25)

# Normal estimation
cloud.add_scalar_field("normals", k_neighbors=k_neighbors_25)

# Visualization
print(cloud.points)

Next, we will compute the characteristics based on eigenvalues

In [ ]:
# Calculation of eigenvalues
eigenvalues = cloud.add_scalar_field("eigen_values", k_neighbors=k_neighbors_25)

# Calculation of local geometric features
cloud.add_scalar_field("curvature", ev=eigenvalues)
cloud.add_scalar_field("omnivariance", ev=eigenvalues)
cloud.add_scalar_field("linearity", ev=eigenvalues)
cloud.add_scalar_field("planarity", ev=eigenvalues)
cloud.add_scalar_field("sphericity", ev=eigenvalues)

# Visualization
print(cloud.points)

To be used in the algorithm, the features must be returned in numpy array format. Also, not all "points" features are useful, we will select the normals and those calculated from the eigenvalues, but not the coordinates or the eigenvalues themselves.

In [ ]:
# Feature selection
train_features = cloud.points[['nx(26)','ny(26)','nz(26)','curvature(26)','omnivariance(26)','linearity(26)','planarity(26)','sphericity(26)',]].to_numpy()

# Visualization
print(train_features)

# Definitions

Before proceeding, let's define some functions.

We have seen step by step how to read the input data, split it and extract its features. These features are the ones used for training, but they are also necessary for future classification, so it is necessary for each sample to extract these features and generate a data matrix whose attributes correspond to the training order. To avoid repeating code each time we load a data matrix, we will define two functions. The first one will separate the data. The second one will extract the features.

In [ ]:
def separar_input(input_matrix):
    coord = pd.DataFrame(list(zip(input_matrix[:,0],input_matrix[:,1],input_matrix[:,2])))  
    coord.columns =['x', 'y', 'z']
    labels = input_matrix[:,3]
    return coord, labels

In [ ]:
def extraer_features(coord):
    # Create point cloud
    cloud = PyntCloud(coord)
    # Calculate neighbors
    k_neighbors_25 = cloud.get_neighbors(k=25)
    # Calculate and add normals
    cloud.add_scalar_field("normals", k_neighbors=k_neighbors_25)
    # Calculate and add eigenvalues
    eigenvalues = cloud.add_scalar_field("eigen_values", k_neighbors=k_neighbors_25)
    # Calculate and add other geometrical characteristics
    cloud.add_scalar_field("curvature", ev=eigenvalues)
    cloud.add_scalar_field("omnivariance", ev=eigenvalues)
    cloud.add_scalar_field("linearity", ev=eigenvalues)
    cloud.add_scalar_field("planarity", ev=eigenvalues)
    cloud.add_scalar_field("sphericity", ev=eigenvalues)
    # Transform point dataframe to feature nparray (in order)
    features = cloud.points[['nx(26)','ny(26)','nz(26)','curvature(26)','omnivariance(26)','linearity(26)','planarity(26)','sphericity(26)',]].to_numpy()
    return features

In [ ]:
# Separate input data
train_coord,train_labels = separar_input(train_data)

# Extract features
train_features = extraer_features(train_coord)

# Algorithm training

In this step we get to the core of the algorithm. Once all the data are prepared we proceed to the training. In this practice we are going to train and use the two most used algorithms nowadays: 
- Support Vector Machine. Given a set of points, in which each of them belongs to one of two possible categories, an SVM-based algorithm builds a model capable of predicting whether a new point (whose category we do not know) belongs to one category or the other.

<center> <img src="img/svm.png"></center>
<center>Fuente: https://en.wikipedia.org/wiki/Support-vector_machine</center>

- Random Forest. It is a combination of predictor trees such that each tree depends on the values of a random vector tested independently and with the same distribution for each of them. The result of the classification is the class predicted by the majority of all trees.

<center> <img src="img/rf.png"></center>
<center>Fuente: https://es.wikipedia.org/wiki/Random_forest</center>

Once all the work has been done, all you have to do is give the data to the algorithm to train.


In [ ]:
# Define SVM classifier
clf_svm = svm.SVC()

# Train (this may take several minutes)
clf_svm.fit(train_features, train_labels)

In [ ]:
# Define RF classifier
clf_rf = RandomForestClassifier()

#Train (this should go faster than the previous one)
clf_rf.fit(train_features, train_labels)

# Analysis of results through metrics

Metrics let us know how good our trained algorithm is with respect to reality. If metrics are applied on the training data we will know if our algorithm has been able to learn from the data provided. But to obtain reliable and verifiable results about the good performance of the algorithm we will have to test it on data other than those used for training. By default, the sklearn library offers functions to use the most common metrics:

<center> <img src="img/matcon.jpg"></center>
<center>Fuente: Confusion Matrix - Applied Deep Learning with Keras</center>


<center> <img src="img/met.png"></center>
<center>Fuente: https://www.researchgate.net/</center>

First, let's apply the metrics to the training data to observe which algorithm has learned better, i.e. identified and extracted more information.

In [ ]:
# We classify the points (through the features) (This may take a few minutes)

# With the SVM
train_predictions_SVM = clf_svm.predict(train_features)

# With the RF
train_predictions_RF = clf_rf.predict(train_features)

In [ ]:
# Calculation of metrics for the SVM algorithm

# Here we define the ground truth data.
y_test = train_labels

# Here we define the data we have calculated.
y_pred = train_predictions_SVM

# Calculates and displays the confusion matrix
print(confusion_matrix(y_test,y_pred))

# Calculates and displays statistics by class
print(classification_report(y_test,y_pred))

# Calculates and displays global statistics
print(accuracy_score(y_test, y_pred))

In [ ]:
# Cálculo de métricas para el algoritmo RF

# Here we define the ground truth data.
y_test = train_labels

# Here we define the data we have calculated.
y_pred = train_predictions_RF

# Calculates and displays the confusion matrix
print(confusion_matrix(y_test,y_pred))

# Calculates and displays statistics by class
print(classification_report(y_test,y_pred))

# Calculates and displays global statistics
print(accuracy_score(y_test, y_pred))

As can be seen in the metrics above, both classifiers were trained correctly and obtained high success rates, with those of the RF being higher than those of the SVM. At least that is what can be deduced from the predictions with the training data. For a stricter evaluation, we will now use the test data. 

In [ ]:
# Data reading
test_data = np.loadtxt("Nubes/000079.txt", delimiter=' ')

# Separate input data
test_coord,test_labels = separar_input(test_data)

# Extract features
test_features = extraer_features(test_coord)

In [ ]:
# We classify the points (through the features) (This may take a few minutes)

# SVM
test_predictions_SVM = clf_svm.predict(test_features)

# RF
test_predictions_RF = clf_rf.predict(test_features)

In [ ]:
# Calculation of metrics for the SVM algorithm

# Here we define the ground truth data.
y_test = test_labels

# Here we define the data we have calculated.
y_pred = test_predictions_SVM

# Calculates and displays the confusion matrix
print(confusion_matrix(y_test,y_pred))

# Calculates and displays statistics by class
print(classification_report(y_test,y_pred))

# Calculates and displays global statistics
print(accuracy_score(y_test, y_pred))

In [ ]:
# Calculation of metrics for the RF algorithm

# Here we define the ground truth data.
y_test = test_labels

# Here we define the data we have calculated.
y_pred = test_predictions_RF

# Calculates and displays the confusion matrix
print(confusion_matrix(y_test,y_pred))

# Calculates and displays statistics by class
print(classification_report(y_test,y_pred))

# Calculates and displays global statisticss
print(accuracy_score(y_test, y_pred))

We can observe that the golbal accuracies are very similar between both classifiers. For this reason it is important to test the results with data independent of training. Comparing the training and testing data, we see that both algorithms have a strong tendency to overfitting, being more noticeable with the RF that goes from an accuracy of 99% to 76%. 

# Visualization of the classification

An advantage of point clouds is that we can interpret the results based on visualization as well, since they are geometric data. If we use Artificial Intelligence to solve purely numerical problems, such a clear visualization of the data is not possible. The visualization of the data is very relevant to be able to identify possible failures that go unnoticed in the metrics.

Finally, we proceed to export the results in a point cloud to visualize in CloudCompare. The exported cloud will have the following columns: 
- 3 columns of coordinates
- 1 column of labels (ground truth)
- 1 column of SVM prediction
- 1 RF prediction column

Are any problems detected that were not detected in the metrics? Are the results as similar as the metrics suggest? Which classes have been classified better? Being able to visualize the problems, can you think of any better solution than the one proposed with the metrics alone?

In [ ]:
# Export
# Definition of the path and file name
ruta = "Nubes/00000079_predicted.txt"

#Data selection 
datos = np.column_stack((test_data,test_predictions_SVM,test_predictions_RF))

# Saved
np.savetxt(ruta,datos,delimiter=' ') 